<a href="https://colab.research.google.com/github/jungfrau70/encore-4team/blob/main/masks_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Masks Detection Web Service

https://masks-detection-13237.web.app

https://github.com/ohyicong/masksdetection.git

# Download Code from GITHUB

In [1]:
!git clone https://github.com/ohyicong/masksdetection.git/

fatal: destination path 'masksdetection' already exists and is not an empty directory.


## STEP 1. import

In [2]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import tensorflow as tf

## STEP 2. 전처리 (pre-processing)

In [3]:
print(os.getcwd())
!ls -al masksdetection

/content
total 10904
drwxr-xr-x 13 root root     4096 May 29 15:13 .
drwxr-xr-x  1 root root     4096 May 29 14:48 ..
-rw-r--r--  1 root root       93 May 29 15:11 checkpoint
-rw-r--r--  1 root root     5788 May 29 14:49 create_augmented_dataset.py
drwxr-xr-x  4 root root     4096 May 29 14:49 dataset
-rw-r--r--  1 root root     4848 May 29 14:49 detect_mask_video.py
-rw-r--r--  1 root root     1059 May 29 14:49 detect_mask_video.spec
drwxr-xr-x  2 root root     4096 May 29 14:49 face_detector
drwxr-xr-x  8 root root     4096 May 29 14:49 .git
-rw-r--r--  1 root root       28 May 29 14:49 .gitignore
drwxr-xr-x  2 root root     4096 May 29 14:49 images
drwxr-xr-x  2 root root     4096 May 29 14:49 .ipynb_checkpoints
drwxr-xr-x 12 root root     4096 May 29 15:13 masksdetection
drwxr-xr-x  2 root root     4096 May 29 14:49 model
-rw-r--r--  1 root root 11043490 May 29 15:11 my_checkpoint.ckpt.data-00000-of-00001
-rw-r--r--  1 root root    17711 May 29 15:11 my_checkpoint.ckpt.index
drwxr-

In [4]:
import os
os.chdir('masksdetection')
!ls -al .

total 10904
drwxr-xr-x 13 root root     4096 May 29 15:13 .
drwxr-xr-x  1 root root     4096 May 29 14:48 ..
-rw-r--r--  1 root root       93 May 29 15:11 checkpoint
-rw-r--r--  1 root root     5788 May 29 14:49 create_augmented_dataset.py
drwxr-xr-x  4 root root     4096 May 29 14:49 dataset
-rw-r--r--  1 root root     4848 May 29 14:49 detect_mask_video.py
-rw-r--r--  1 root root     1059 May 29 14:49 detect_mask_video.spec
drwxr-xr-x  2 root root     4096 May 29 14:49 face_detector
drwxr-xr-x  8 root root     4096 May 29 14:49 .git
-rw-r--r--  1 root root       28 May 29 14:49 .gitignore
drwxr-xr-x  2 root root     4096 May 29 14:49 images
drwxr-xr-x  2 root root     4096 May 29 14:49 .ipynb_checkpoints
drwxr-xr-x 12 root root     4096 May 29 15:13 masksdetection
drwxr-xr-x  2 root root     4096 May 29 14:49 model
-rw-r--r--  1 root root 11043490 May 29 15:11 my_checkpoint.ckpt.data-00000-of-00001
-rw-r--r--  1 root root    17711 May 29 15:11 my_checkpoint.ckpt.index
drwxr-xr-x  2 r

In [6]:
# construct the argument parser and parse the arguments
dataset_path=os.getcwd()+"//dataset"
model_path=os.getcwd()+"//model//mask_model"
plot_path=os.getcwd()+"//plot"

In [7]:
training_datagen = ImageDataGenerator(
    rescale=1. / 255,
    # 위의 옵션 값들을 보고 적절히 대입하여 줍니다.
    rotation_range=20,
    zoom_range=0.15,  
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.15, 
    horizontal_flip=True, 
    fill_mode='nearest', 
    validation_split=0.2
    )

In [8]:
TRAINING_DIR = "dataset/"

In [9]:
training_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                            target_size=(224, 224),
                                                            color_mode='rgb', classes=None,
                                                            class_mode='categorical',
                                                            batch_size=128,                                                            
                                                            subset='training', 
)

Found 1131 images belonging to 2 classes.


In [10]:
validation_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                            target_size=(224, 224),
                                                            color_mode='rgb', classes=None,
                                                            class_mode='categorical',
                                                            batch_size=128,                                                            
                                                            subset='validation', 
                                                            )

Found 282 images belonging to 2 classes.


## STEP 3. 모델 정의 (Sequential)


In [11]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

## STEP 4. optimizer, loss

In [13]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [14]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

[INFO] compiling model...


## STEP 5. ModelCheckpoint

In [15]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

## STEP 6. 학습 (fit)

In [16]:
H = model.fit(training_generator,
          epochs=BS, 
          verbose=1, 
          callbacks=[checkpoint],
          validation_data=(validation_generator)
          )

Epoch 1/32
9/9 [==============================] - 23s 2s/step - loss: 0.8473 - acc: 0.5712 - val_loss: 0.3482 - val_acc: 0.9043

Epoch 00001: val_loss improved from inf to 0.34817, saving model to my_checkpoint.ckpt
Epoch 2/32
9/9 [==============================] - 15s 2s/step - loss: 0.3692 - acc: 0.8311 - val_loss: 0.1480 - val_acc: 0.9787

Epoch 00002: val_loss improved from 0.34817 to 0.14803, saving model to my_checkpoint.ckpt
Epoch 3/32
9/9 [==============================] - 16s 2s/step - loss: 0.1763 - acc: 0.9416 - val_loss: 0.0977 - val_acc: 0.9823

Epoch 00003: val_loss improved from 0.14803 to 0.09765, saving model to my_checkpoint.ckpt
Epoch 4/32
9/9 [==============================] - 15s 2s/step - loss: 0.1176 - acc: 0.9637 - val_loss: 0.0729 - val_acc: 0.9823

Epoch 00004: val_loss improved from 0.09765 to 0.07293, saving model to my_checkpoint.ckpt
Epoch 5/32
9/9 [==============================] - 15s 2s/step - loss: 0.0788 - acc: 0.9805 - val_loss: 0.0614 - val_acc: 0.9

## STEP 7. 학습 완료 후 Load Weights (ModelCheckpoint)

In [17]:
model.load_weights(checkpoint_path)

## STEP 8. 예측 (Predict)

In [18]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] evaluating network...


NameError: ignored

## STEP 9. 리포트

In [19]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot_path)

NameError: ignored

## STEP 10. 모델 저장

In [20]:
# serialize the model to disk
print("[INFO] saving mask detector model... path: %s"%(model_path+".h5"))
model.save(model_path+".h5")

[INFO] saving mask detector model... path: /content/masksdetection//model//mask_model.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
